# Creating a random Network

### Represented by an adjacency matrix

In [839]:
# setting the number of nodes 
nodes = 1000

Max number of edges

In [840]:
nodes * (nodes-1) / 2

499500.0

Seting up a (p) probability that will likely result in a random network with 10,000 edges 

I will confirm that the network has 10,000 edges after

In [841]:
p = 0.006

An estimate of how many edges this network could have

In [842]:
p * nodes * (nodes-1) / 2

2997.0

Creating the adjacency matrix structure for the random network

In [843]:
network_matrix = []

for i in range(nodes):
    row = []
    for j in range(nodes):
        row.append(0)
    network_matrix.append(row)

Creating the edges between pairs of nodes

In [844]:
# !pip install numpy

In [845]:
from numpy import random

# iterating through every pair of nodes one time
# pairs i,j are being considered equal to pair j,i
for i in range(len(network_matrix)):
    for j in range(i+1, len(network_matrix[0])):
        # if the i node is different from j node, therefore creating a pair of nodes
        if i != j:
            random_prob = random.rand()
            if random_prob < p:    
                # add an edge
                # is a non directed graph, so the two nodes have the connection
                network_matrix[i][j] = 1
                network_matrix[j][i] = 1
 

In [846]:
def count_edges(edge_list):
    count = 0
    # iterating through every unique pair combination
    for i in range(len(edge_list)):
        for j in range(i+1, len(edge_list[0])):
            if edge_list[i][j] == 1:
                count += 1
    return count

Real number of edges

In [847]:
network_edges = count_edges(network_matrix)
print(network_edges)

2983


### Using the adjacency list to create a graph from graph-tools

In [848]:
from graph_tool.all import *
import scipy

In [849]:
ug_random_network = Graph(scipy.sparse.lil_matrix(network_matrix),directed=False)

## Saving the network as a .tsv file

In [850]:
filepath = "edge-list.tsv"

with open(filepath, 'w') as f:
    for i in range(len(network_matrix)):
        for j in range(i+1, len(network_matrix[1])):
            if network_matrix[i][j] == 1:
                f.write(f"{i}\t{j}\n")



## Saving the network as a METIS file

In [851]:
filepath = "network.metis"

with open(filepath, 'w') as f:
    f.write(f"{nodes} {network_edges}\n")
    for i in range(len(network_matrix)):
        neighboors = []
        for j in range(len(network_matrix[0])):
            if j != i:
                neighboors.append(str(j+1))
        neighboors.append("\n")
        f.write(" ".join(neighboors))



## Saving the network as a graphML file

In [852]:
path_file = "network.grapml"
ug_random_network.save(path_file, fmt="graphml")

## Degree Distribuiton

In [853]:
#!pip install matplotlib

In [854]:
# lisitng as the nodes degrees
degree_list = ug_random_network.get_out_degrees(ug_random_network.get_vertices())
degree_list.sort()
degree_list

array([ 0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4

Creating the degree distribuition figure

In [855]:
import matplotlib.pyplot as plt

In [856]:
plt.figure(figsize=(10,7),dpi=1000)
plt.hist(degree_list, bins= 30, edgecolor = 'black')
plt.xticks(degree_list)
plt.title("Degree Distribuition")
plt.xlabel("Degree")
plt.ylabel("Number of nodes")
plt.show() 

## Number of isolated nodes

In [857]:
# count how many nodes have 0 degree, from the degrees list
number_isolated_nodes = 0

for dg in degree_list:
    if dg == 0:
        number_isolated_nodes += 1

In [858]:
print(number_isolated_nodes)

2


## Number of connected components

In [859]:
comp, hist =  label_components(ug_random_network, directed=False)

In [860]:
# hist represents "Histogram of component labels""
hist

array([998,   1,   1], dtype=uint64)

So its lenght will represent the total connected components

In [861]:
len(hist)

3